In [1]:
import hvplot.xarray
import numpy as np
import scipy.stats
import xarray as xr

import climepi
import climepi.climdata as climdata
import climepi.epimod as epimod

In [2]:
ds_clim = climdata.get_example_dataset("isimip_london")
ds_clim["time"].attrs["long_name"] = "Time"
climepi_modes = ds_clim.climepi.modes
ds_clim = (
    ds_clim.drop_vars(["lon_bnds", "lat_bnds"])
    .squeeze()
    .sel({"scenario": ["ssp126", "ssp370", "ssp585"]})
)
ds_clim.climepi.modes = climepi_modes

In [3]:
ds_clim_ym = ds_clim.climepi.yearly_average()
ds_clim_ym.climepi.plot_time_series("temperature", by=["scenario"])

BokehModel(combine_events=True, render_bundle={'docs_json': {'6f588dfd-847a-4f3f-85d0-084a9aaffa21': {'version…

In [4]:
ecol_niche_model = epimod.ecolniche.get_kaye_model()
ds_clim.epimod.model = ecol_niche_model
ds_epi = ds_clim.epimod.run_model()
ds_months_suitable = ds_epi.epimod.months_suitable()
ds_months_suitable.climepi.plot_time_series(by=["scenario"])

BokehModel(combine_events=True, render_bundle={'docs_json': {'80bf77fa-b1de-425e-875c-dec5e190ad83': {'version…

In [5]:
ds_ym = xr.merge([ds_clim_ym, ds_months_suitable])
ds_ym

<xarray.Dataset>
Dimensions:          (scenario: 3, model: 10, time: 86, bnds: 2)
Coordinates:
  * scenario         (scenario) object 'ssp126' 'ssp370' 'ssp585'
    realization      int64 0
    lat              float64 51.75
    lon              float64 -0.25
  * model            (model) object 'canesm5' 'cnrm-cm6-1' ... 'ukesm1-0-ll'
  * time             (time) object 2015-07-02 12:00:00 ... 2100-07-02 12:00:00
Dimensions without coordinates: bnds
Data variables:
    temperature      (scenario, model, time) float64 dask.array<chunksize=(1, 1, 86), meta=np.ndarray>
    time_bnds        (time, bnds) object 2015-01-01 00:00:00 ... 2101-01-01 0...
    precipitation    (scenario, model, time) float64 dask.array<chunksize=(1, 1, 86), meta=np.ndarray>
    months_suitable  (scenario, model, time) float64 dask.array<chunksize=(1, 1, 86), meta=np.ndarray>
Attributes:
    title:        ISIMIP3b bias-adjusted climate input data
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    summary:      CMIP6 daily output data bias-adjusted and statistically dow...
    references:   Lange (2019) <https://doi.org/10.5194/gmd-12-3055-2019> and...

In [6]:
ds_ym_stat = ds_ym.climepi.ensemble_stats(
    conf_level=90,
    estimate_internal_variability=True,
)
ds_ym_stat

<xarray.Dataset>
Dimensions:          (scenario: 3, model: 10, time: 86, ensemble_stat: 5)
Coordinates:
  * scenario         (scenario) object 'ssp126' 'ssp370' 'ssp585'
  * model            (model) object 'canesm5' 'cnrm-cm6-1' ... 'ukesm1-0-ll'
    lat              float64 51.75
    lon              float64 -0.25
  * time             (time) object 2015-07-02 12:00:00 ... 2100-07-02 12:00:00
  * ensemble_stat    (ensemble_stat) <U5 'mean' 'var' 'std' 'lower' 'upper'
Data variables:
    temperature      (ensemble_stat, time, scenario, model) float64 dask.array<chunksize=(1, 86, 1, 10), meta=np.ndarray>
    precipitation    (ensemble_stat, time, scenario, model) float64 dask.array<chunksize=(1, 86, 1, 10), meta=np.ndarray>
    months_suitable  (ensemble_stat, time, scenario, model) float64 dask.array<chunksize=(1, 86, 1, 10), meta=np.ndarray>
Attributes:
    title:        ISIMIP3b bias-adjusted climate input data
    institution:  Potsdam Institute for Climate Impact Research (PIK)
    project:      Inter-Sectoral Impact Model Intercomparison Project phase 3...
    contact:      ISIMIP cross-sectoral science team <info@isimip.org> <https...
    summary:      CMIP6 daily output data bias-adjusted and statistically dow...
    references:   Lange (2019) <https://doi.org/10.5194/gmd-12-3055-2019> and...

In [7]:
data_var_plot = "months_suitable"

In [8]:
p1 = ds_ym_stat.sel(ensemble_stat="mean").climepi.plot_time_series(
    data_var_plot, by=["scenario"]
)
p2 = ds_ym.climepi.plot_time_series(data_var_plot, by=["scenario"])
p1 * p2

BokehModel(combine_events=True, render_bundle={'docs_json': {'43032d92-65c7-425e-ba21-7a94f77d1bc5': {'version…

In [9]:
ds_ym.climepi.plot_var_decomp(
    data_var=data_var_plot, estimate_internal_variability=True, fraction=False
)

:NdOverlay   [Uncertainty type]
   :Area   [time]   (value,Baseline)

In [10]:
ds_ym.climepi.plot_ci_plume(
    data_var=data_var_plot, conf_level=90, estimate_internal_variability=True, alpha=0.4
)

:Overlay
   .Area.Scenario  :Area   [time]   (lower,upper)
   .Area.Model     :Area   [time]   (lower,upper)
   .Area.Internal  :Area   [time]   (lower,upper)
   .Curve.Baseline :Curve   [time]   (months_suitable)

In [11]:
scenario_ex = "ssp370"
model_ex = "ipsl-cm6a-lr"
ds_ym.sel(scenario=scenario_ex).climepi.plot_time_series(
    data_var=data_var_plot
) * ds_ym.sel(scenario=scenario_ex).climepi.plot_ci_plume(
    data_var=data_var_plot, conf_level=90, estimate_internal_variability=True, alpha=0.4
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'01c3bb6c-8083-43d5-b117-e4b827fd8a9f': {'version…

In [14]:
ds_ym.sel(model=model_ex).climepi.plot_time_series(
    data_var=data_var_plot, by="scenario"
) * ds_ym.sel(model=model_ex).climepi.plot_ci_plume(
    data_var=data_var_plot,
    conf_level=90,
    estimate_internal_variability=True,
    alpha=0.4,
)

:Overlay
   .NdOverlay.I    :NdOverlay   [scenario]
      :Curve   [time]   (months_suitable)
   .Area.Scenario  :Area   [time]   (lower,upper)
   .Area.Internal  :Area   [time]   (lower,upper)
   .Curve.Baseline :Curve   [time]   (months_suitable)

In [17]:
ds_ym.sel(scenario=scenario_ex).climepi.plot_time_series(
    data_var=data_var_plot, by="model"
) * ds_ym.sel(scenario=scenario_ex).climepi.plot_ci_plume(
    data_var=data_var_plot,
    conf_level=90,
    estimate_internal_variability=False,
    alpha=0.4,
)

:Overlay
   .NdOverlay.I    :NdOverlay   [model]
      :Curve   [time]   (months_suitable)
   .Area.Model     :Area   [time]   (lower,upper)
   .Curve.Baseline :Curve   [time]   (months_suitable)